In [3]:
import pandas as pd


In [2]:
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [5]:
plant_df = pd.read_csv('plant.csv')  # read the plant data from the csv file
rock_df = pd.read_csv('rock.csv')  # read the rock data from the csv file